In [5]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np


In [6]:
## load the ANN train model , scaler picle , onehot file

model = load_model('model.h5')

# load the encoder and scaler

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('label_encoder_geo.pkl', 'rb') as file:
    label_encoder_geo = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [24]:
input_data = {
    'CreditScore' : 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 30000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary' : 50000
}

In [25]:
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(
    geo_encoded,
    columns=label_encoder_geo.get_feature_names_out(['Geography'])
)
geo_encoded_df

/home/kanbs/Desktop/GenAi/Project/ANN Project/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [26]:
input_df = pd.DataFrame([input_data])

input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,30000,2,1,1,50000


In [27]:
# Encode categorical variable

input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,30000,2,1,1,50000


In [28]:
input_df = pd.concat(
    [input_df.drop(['Geography'], axis=1), geo_encoded_df],
    axis=1
)

input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,30000,2,1,1,50000,1.0,0.0,0.0


In [29]:
## scaling the input data

input_scaled = scaler.transform(input_df)

input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.73814087,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [30]:
## for pridection

pre = model.predict(input_scaled)

pre

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step


array([[0.02112624]], dtype=float32)

In [31]:
pre_proba = pre[0][0]

In [32]:
if pre_proba > .5:
    print('the customer is likey to churn')
else:
    print('the customer is not likely to chrun')

the customer is not likely to chrun
